# 1 Dataset protein preprocess for submit
convert all proteins in lit-pcba dataset to FASTA seq

In [1]:
import os
import subprocess
import numpy as np
from subprocess import Popen, PIPE
from pathlib import Path
dataset_dir = Path('../../../Dataset/GLAM/raw/lit_pcba_raw/')
assert dataset_dir.exists()

# 2Submit and results download
Submit to http://raptorx.uchicago.edu/ContactMap/ and Download all zip file from the results  
or drug.ai.tencent.com

 ...

# 3. Raw contact map to {seq:matrix} dict
{'ACKFIDSAKLFKDLSJAKLFJ':'MATRIX torch.tensor'}

### File Glob

In [2]:
import torch
from tqdm import tqdm
from pathlib import Path

def read_probs(path, mean_prob=False):
    # fh = open(filename, 'r')
    # content = [line.strip() for line in list(fh)]
    # fh.close()
    # print(content)
    with open(path, 'r') as f:
        content = f.readlines()

    assert len(content) >= 5  # '1. the input file contains fewer than 5 lines'

    seq = ""
    infos = {}
    probs = []

    for line in content:
        # print(line)
        if 'SEQ' in line:
            seq += line.split()[-1]
            continue
        if line.startswith('PFRMAT') or line.startswith('TARGET') or line.startswith('AUTHOR') or \
                line.startswith('METHOD') or line.startswith('RMODE') or line.startswith('RMODE') or \
                line.startswith('MODEL') or line.startswith('REMARK') or line.startswith('END'):
            infos[line.split()[0]] = line.split()[1:]
            continue

        columns = line.split()

        if len(columns) >= 3:
            indices = [int(x) for x in columns[0:2]]
            prob = np.float32(columns[2])
            # if mean_prob:
            #     prob = np.mean([float(x) for x in columns[-10:-1:2]])  # todo: need to check when using

            assert 0 <= prob <= 1  # 'The contact prob shall be between 0 and 1: '
            # assert 0 < c < 20  # 'The distance shall be between 0 and 20: '
            assert indices[0] < indices[1]  # 'The first index in a residue pair shall be smaller than the 2nd one:'

            if indices[0] < 1 or indices[0] > len(seq) or indices[1] < 1 or indices[1] > len(seq):
                print('The residue index in the following line is out of range: \n', line)
                return None
            probs.append(indices + [prob])
        else:
            print('The following line in the input file has an incorrect format: ')
            print(line)
            return None
    return probs, seq, infos


def load_contactmap(path, thre=0.1):
    # 0.1  thre to keep 2988/30000 prob data of a 894 AAs protein
    # 0.05 thre to keep 4700/30000 prob data
    # 0.3  thre to keep 1505/30000 prob data
    probs, seq, infos = read_probs(path)
    contactmap = np.zeros((len(seq), len(seq)), dtype=np.float32)
    for p in probs:
        if p[2] >= thre:
            contactmap[p[0] - 1, p[1] - 1] = p[2]
            contactmap[p[1] - 1, p[0] - 1] = p[2]
    return contactmap, seq, infos

In [6]:
pro_dm_dir = dataset_dir/'raptorx_pred'/'contact_8.5'

In [7]:
protein_maps_dict = {}  # 574
for path in tqdm(pro_dm_dir.glob('*.contactmap.txt')):
    contact_map, seq, infos = load_contactmap(path)
    protein_maps_dict[seq] = contact_map

5it [00:01,  4.20it/s]


In [8]:
torch.save(protein_maps_dict, pro_dm_dir/'protein_maps_dict.ckpt')

In [9]:
protein_maps_dict = torch.load(pro_dm_dir/'protein_maps_dict.ckpt')
# protein_maps_dict.keys()